In [1]:
import os
import random
import warnings
warnings.filterwarnings("ignore")
import import_ipynb
import utils
from utils import train_test_split

importing Jupyter notebook from utils.ipynb


In [2]:
src = 'Dataset/PetImages/'

# create the train/test folders if it does not exists already
if not os.path.isdir(src+'train/'):
    train_test_split(src)

In [3]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [4]:
# Define hyperparameters
INPUT_SIZE = 128 #Change this to 48 if the code is taking too long to run
BATCH_SIZE = 16
STEPS_PER_EPOCH = 200
EPOCHS = 3

vgg16 = VGG16(include_top = False, weights = 'imagenet', input_shape = (INPUT_SIZE,INPUT_SIZE, 3))

In [5]:
# Freeze the pre-trained layers
for layer in vgg16.layers:
    layer.trainable = False

In [6]:
# Add a fully connected layer with 1 node at the end 
input_ = vgg16.input
output_ = vgg16(input_)
last_layer = Flatten(name = 'flatten')(output_)
last_layer = Dense(1, activation = 'sigmoid')(last_layer)
model = Model(input = input_, output = last_layer)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

training_data_generator = ImageDataGenerator(rescale = 1./255)
testing_data_generator = ImageDataGenerator(rescale = 1./255)

training_set = training_data_generator.flow_from_directory(src + 'Train/',
                                                target_size = (INPUT_SIZE, INPUT_SIZE),
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'binary')

test_set = testing_data_generator.flow_from_directory(src + 'Test/',
                                             target_size = (INPUT_SIZE, INPUT_SIZE),
                                             batch_size = BATCH_SIZE,
                                             class_mode = 'binary')

model.fit_generator(training_set, steps_per_epoch = STEPS_PER_EPOCH, epochs = EPOCHS, verbose = 1)

score = model.evaluate_generator(test_set, steps = 100)

for idx, metric in enumerate(model.metrics_names):
    print("{}: {}".format(metric, score[idx]))

Found 19997 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/3
200/200 [==============================] - 306s 2s/step - loss: 0.3817 - accuracy: 0.8203
Epoch 2/3
200/200 [==============================] - 298s 1s/step - loss: 0.2793 - accuracy: 0.8781
Epoch 3/3
200/200 [==============================] - 295s 1s/step - loss: 0.2693 - accuracy: 0.8897
loss: 0.47205138206481934
accuracy: 0.8762500286102295
